In [5]:
#from MuZeroParallel import MuZero as MuZeroP

from MuZero import MuZero

from GridWorldOption import GridWorldOption


from GridWorld import GridWorld

from mcts.MCTS import MCTS
import numpy as np
import matplotlib.pyplot as plt

from random import choice
from copy import deepcopy
import time

import pickle
import pandas as pd

from matplotlib.pyplot import figure
import itertools
from tqdm import tqdm


MAP_NAME = './maps/door.map'

options = [
      GridWorldOption((0, 0),  {'all'}, 0, 0),
      GridWorldOption((0, 0),  {'all'}, 1, 1),
      GridWorldOption((0, 0),  {'all'}, 2, 2),
      GridWorldOption((0, 0),  {'all'}, 3, 3)
]

with open(MAP_NAME) as f:
    the_map = f.read()
    
first_room_pos = [(i,j) for i in range(6) for j in range(8)]
second_room_pos = [(i,j) for i in range(6) for j in range(9, 15)]
third_room_pos = [(i,j) for i in range(7,13) for j in range(8)]
fourth_room_pos = [(i,j) for i in range(7,13) for j in range(9,15)]

full_options = [
  # primitives, (0,0) is meaningless
  GridWorldOption((0, 0),   {'all'}, 0, 0),
  GridWorldOption((0, 0),   {'all'}, 1, 1),
  GridWorldOption((0, 0),   {'all'}, 2, 2),
  GridWorldOption((0, 0),   {'all'}, 3, 3),
  GridWorldOption((3,8),  set(first_room_pos + second_room_pos + [(6,3)] + [(6,13)]), 4),
  GridWorldOption((6,3),  set(first_room_pos + third_room_pos + [(3,8)] + [(11,8)]), 5),
  GridWorldOption((6,13), set(second_room_pos + fourth_room_pos + [(3,8)] + [(11,8)]), 6),
  GridWorldOption((11,8), set(third_room_pos + fourth_room_pos + [(6,3)] + [(6,13)]), 7),
]    

env = GridWorld(the_map)

G_model = {}
for position in tqdm(env.possible_positions):
    for option in full_options:
        if option.is_valid(position):
            
            test_env = GridWorld(the_map)
            test_env.cur_x = position[0]
            test_env.cur_y = position[1]
            rewards = []
            while True:
                action, should_break = option.get_action(test_env)
                
                if action == -1:
                    break
                
                next_position, r, _ = test_env.step(action)
                rewards.append(r)
                
                if should_break:
                    break
                
            G_model[(position, option)] = (next_position, rewards)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 172/172 [00:00<00:00, 2675.55it/s]


In [6]:
first_room_pos = [(i,j) for i in range(6) for j in range(8)]
second_room_pos = [(i,j) for i in range(6) for j in range(9, 15)]
third_room_pos = [(i,j) for i in range(7,13) for j in range(8)]
fourth_room_pos = [(i,j) for i in range(7,13) for j in range(9,15)]

options = [
  # primitives, (0,0) is meaningless
  GridWorldOption((0, 0),   {'all'}, 0, 0),
  GridWorldOption((0, 0),   {'all'}, 1, 1),
  GridWorldOption((0, 0),   {'all'}, 2, 2),
  GridWorldOption((0, 0),   {'all'}, 3, 3),
  GridWorldOption((3,8),  set(first_room_pos + second_room_pos + [(6,3)] + [(6,13)]), 4),
  GridWorldOption((6,3),  set(first_room_pos + third_room_pos + [(3,8)] + [(11,8)]), 5),
  GridWorldOption((6,13), set(second_room_pos + fourth_room_pos + [(3,8)] + [(11,8)]), 6),
  GridWorldOption((11,8), set(third_room_pos + fourth_room_pos + [(6,3)] + [(6,13)]), 7),
]

with open(MAP_NAME) as f:
    the_map = f.read()
    
env = GridWorld(the_map)
mu_opt = MuZero(env, options)

mu_opt.g.model_table = G_model


In [7]:
s0 = env.reset()

mcts = MCTS(s0, mu_opt.f, mu_opt.g, mu_opt.options)
mcts.run_sim(100)

array([0.13, 0.3 , 0.39, 0.1 , 0.03, 0.05, 0.  , 0.  ])

In [4]:
mcts.info()

Expanded:  True
Leaf:  False
hidden state (1, 1)
Option 0
N(s, o) 18
Q(s, o) -5.4047424680388865
P(s, o) 0.16666666666666666
Q'(s, o):  0.5346861149074393
UCT exploration:  0.5263157894736842
Prior regulation:  1.2551262642394618
U(s, o) 0.644784910016164
Option 1
N(s, o) 37
Q(s, o) -4.8710413041324365
P(s, o) 0.16666666666666666
Q'(s, o):  0.5910659291321514
UCT exploration:  0.2631578947368421
Prior regulation:  1.2551262642394618
U(s, o) 0.6461153266865137
Option 2
N(s, o) 22
Q(s, o) -5.600774385745455
P(s, o) 0.16666666666666666
Q'(s, o):  0.5139774414964373
UCT exploration:  0.43478260869565216
Prior regulation:  1.2551262642394618
U(s, o) 0.6049286200645142
Option 3
N(s, o) 15
Q(s, o) -5.511914011993334
P(s, o) 0.16666666666666666
Q'(s, o):  0.5233645886546197
UCT exploration:  0.625
Prior regulation:  1.2551262642394618
U(s, o) 0.6541069078462303
Option 4
N(s, o) 3
Q(s, o) -9.688313943599999
P(s, o) 0.16666666666666666
Q'(s, o):  0.08217264792773553
UCT exploration:  2.5
Prior r